In [1]:
import re
import string
import nltk
import regex as re 
import pandas as pd 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
file_path = r"C:\Users\Sujal Karmakar\Desktop\Desktop\Data Analyst\Python\python_data_analytics_project\Theme Finder Using Caption (NLP)\Data\leanbeast_analysis_ready.csv"

df = pd.read_csv(file_path)

#### 1) Basic Cleaning (lower case, remover numbers, emoji, etc)

In [3]:
df_caption = df["caption_text"]

In [4]:
import re

# Your custom list of generic words and hashtags to remove (lowercase, no # for hashtags)
generic_words = set([
    "fitness", "gym", "workout", "fitnessmotivation", "bodybuilding", "training",
    "fit", "muscle", "fitfam", "gymlife", "shredded", "sixpack",
    "fitspo", "personaltrainer", "fitnessmodel", "exercise",
    "comment", "follow", "share", "message", "dm", "whatsapp", "send",
    "will", "i",
    "interested in paid transformation", "now", 
    "amazing", "total", "this is just an act",
    "free diet plan", "zero", "link", "tap", "check", "list"
])

# Optional: split multi-word phrases into words, or just handle them later as whole phrases.

def clean_caption(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    # Lowercase the text for uniformity
    text_lower = text.lower()
    
    # Remove hashtags (#) before splitting, or treat hashtags specially
    # Here, replace hashtags with words without #
    text_lower = re.sub(r'#(\w+)', r'\1', text_lower)
    
    # Remove multiple spaces/newlines
    text_lower = re.sub(r'\s+', ' ', text_lower)
    
    # Remove the generic words from the text
    # To handle multi-word phrases, we can do iterative replacement:
    for phrase in generic_words:
        # Use regex to replace whole words/phrases only (case insensitive)
        # Add word boundaries \b to avoid partial matches
        pattern = r'\b' + re.escape(phrase) + r'\b'
        text_lower = re.sub(pattern, '', text_lower, flags=re.IGNORECASE)
    
    # Clean extra spaces again after removals
    text_lower = re.sub(r'\s+', ' ', text_lower).strip()
    
    return text_lower

# Apply to your caption column
df_caption_cleaned = df_caption.apply(clean_caption)

In [5]:
df_caption_cleaned.sample(15)

385                                    hung curd pure veg
1141    itna chini m shocked ?? abs motivation gymmoti...
1109       spotted a stranger doing wrong rear delt fly …
283                                       lets break fast
934     celebrating anniversary❤️ couple love couplego...
316     jai maa vaishno devi🙏 vaishnodevi jaimatadi de...
429     cholesterol is not ur enemy stop getting fooled ❌
1470    6 pack abs … how many packs you own … 2,4,6,8 ...
1261    following healthy lifestyle is easily achievab...
1396    meh peche hat jaata hu abs motivation gymmotiv...
1442    it’s here! the best sugar substitute of all ti...
829                                 meh hi hu nature ☠️☠️
1541    shoot mode make up by -@pawan_makeover shoot f...
1404    perform.sustain.recover! introducing wellcore,...
1514    hey guys are you ready for a challenge? let’s ...
Name: caption_text, dtype: object

#### 2) Tokenization (removing stop words)

In [6]:
# import nltk

# # Define a custom download directory
# custom_nltk_path = r'C:\Users\Sujal Karmakar\.conda\envs\DS\nltk_data'

# # Download resources to the custom path
## nltk.download('punkt', download_dir=custom_nltk_path)
# nltk.download('punkt_tab', download_dir=custom_nltk_path)
# nltk.download('stopwords', download_dir=custom_nltk_path)

# # Manually add the path to NLTK's search locations
# nltk.data.path.append(custom_nltk_path)

# i had to explicitly download these by specifying files and also punkt was not working for me nltk was demanding punkt_tab so i did that 


#### 3) Creating dictionaries and corpus for LDA (Linear discriminant analysis)

In [7]:
from bertopic import BERTopic

# Make sure df_caption_cleaned is a list of strings (captions)
documents = df_caption_cleaned.tolist()

# Option 1: Directly set nr_topics=12 when initializing the model
topic_model = BERTopic(language="english", nr_topics=10, verbose=True)

# Fit the model on your cleaned caption texts
topics, probs = topic_model.fit_transform(documents)

# Show the topic info (topic number, count, name)
topic_info = topic_model.get_topic_info()
print(topic_info)


# Show the top words for each topic except -1 (noise)
for topic_num in topic_info['Topic']:
    if topic_num != -1:  # -1 is the noise topic
        print(f"\nTopic {topic_num}:")
        print(topic_model.get_topic(topic_num))


2025-06-01 00:19:39,170 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/49 [00:00<?, ?it/s]

2025-06-01 00:19:51,508 - BERTopic - Embedding - Completed ✓
2025-06-01 00:19:51,508 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-01 00:20:13,877 - BERTopic - Dimensionality - Completed ✓
2025-06-01 00:20:13,878 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-01 00:20:13,926 - BERTopic - Cluster - Completed ✓
2025-06-01 00:20:13,927 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-06-01 00:20:13,988 - BERTopic - Representation - Completed ✓
2025-06-01 00:20:13,989 - BERTopic - Topic reduction - Reducing number of topics
2025-06-01 00:20:13,999 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-01 00:20:14,041 - BERTopic - Representation - Completed ✓
2025-06-01 00:20:14,043 - BERTopic - Topic reduction - Reduced number of topics from 29 to 10


   Topic  Count                                         Name  \
0     -1    456                           -1_you_and_body_to   
1      0    544  0_motivation_lifestyle_health_gymmotivation   
2      1    235                       1_sugar_protein_and_is   
3      2     90                         2_paneer_eggs_in_you   
4      3     70                      3_me_919171123324_on_or   
5      4     62    4_wedding_photography_happy_photooftheday   
6      5     33                       5_ice_icebath_the_bath   
7      6     25                6_fasting_hours_autophagy_and   
8      7     19                      7_agree_this_all_action   
9      8     17        8_testosterone_steroids_increase_diet   

                                      Representation  \
0  [you, and, body, to, your, of, the, abs, gains...   
1  [motivation, lifestyle, health, gymmotivation,...   
2  [sugar, protein, and, is, to, of, the, you, fo...   
3  [paneer, eggs, in, you, inbox, and, egg, your,...   
4  [me, 9191711

In [8]:
with open("bertopic_topics.txt", "w", encoding="utf-8") as f:
    for idx, topic in topic_model.get_topic_info().iterrows():
        f.write(f"Topic {topic['Topic']} (Count: {topic['Count']}):\n")
        terms = topic_model.get_topic(topic['Topic'])
        for term, weight in terms:
            f.write(f"  {term}: {weight:.4f}\n")
        f.write("\n")

In [9]:
# Mapping topic IDs to clear, professional names
topic_labels = {
    -1: "Fitness Discussions",
     0: "Body Building & Guidance",
     1: "Sugar & Health Awareness",
     2: "Healthy Recipes & Nutrition",
     3: "Miscellaneous Topics",
     4: "Fitness & Lifestyle",
     5: "Body Recovery & Wellness",
     6: "Blueprint/Recipes (Comment Engagament)",
     7: "Personal Reflections",
     8: "Hydration Tips"
}

# Add the human-readable topic name to the dataframe
df_result = pd.DataFrame({
    "Caption": df_caption,
    "Topic_ID": topics
})
df_result["Topic_Name"] = df_result["Topic_ID"].map(topic_labels)


In [10]:
# Number of samples per topic you want to see
n_samples = 5

# Loop through each topic and print n samples
for topic in df_result["Topic_Name"].unique():
    print(f"\n🔷 Topic: {topic}")
    samples = df_result[df_result["Topic_Name"] == topic]["Caption"].sample(n_samples)
    for i, caption in enumerate(samples, 1):
        print(f"  {i}. {caption}")



🔷 Topic: Body Building & Guidance
  1. Post from 2019 #timesofindia
  2. And we took it SERIOUSLY 😹  #fitcouple #fitness #couplegoals #fitnessmotivation #fit #love #fitfam #workout #motivation #bodybuilding #gym #couple #fitlife #fitgirl #gymlife #healthylifestyle #training #fitnessjourney #fitcouplegoals #fitmom #muscle #fitspo #lifestyle #fitnessmodel #instafit #fitnessaddict #gymmotivation #personaltrainer #healthy #bhfyp
  3. Maa ka pyaar ❤️
  4. Bhai kya kar rahe ho?? @imgauravchander
  5. What happen when you leave sugar for 3 months…  Zero sugar means low carbs ….   #abs #fitness #gym #workout #bodybuilding #fit #fitnessmotivation #motivation #muscle #fitnessmodel #fitfam #gymlife #training #gymmotivation #shredded #instafit #sixpack #model #strong #body #fitnessaddict #fitspo #gains #biceps #lifestyle #cardio #exercise #personaltrainer #health #bodybuilder

🔷 Topic: Miscellaneous Topics
  1. Comment now
  2. Afsoos😔😔
  3. Abs☝️❤️
  4. Har har mahadev💪🏻
  5. Natty💯

🔷 Topic: He

In [11]:
df_result.to_csv("caption_topics_labeled.csv", index=False)
